In [200]:
import pandas as pd
import geopandas as gpd
from matplotlib import pyplot as plt

from matplotlib import colors
import rioxarray
import pyproj
from pysheds.grid import Grid
import numpy as np

import time
from shapely import geometry, ops
from shapely.geometry import Polygon
from rasterstats import zonal_stats
from shapely.geometry.multipolygon import MultiPolygon
from shapely.ops import cascaded_union
from shapely.ops import unary_union
import fiona
import rasterio
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar

import os

## Load MO water body geometries
#### This is a clipped version of the original data:
#### https://data-msdis.opendata.arcgis.com/datasets/MSDIS::mo-2020-wqs-lake-classification-and-use/about

MO_water = gpd.read_file("MO_2020_WQS_Lake_Classification_and_Use.geojson", driver='GeoJSON')
MO_water = MO_water.drop(columns="EVENTDAT").drop(columns="REACHSMDAT").round(0) # Need to drop timestamps for plotting to work
MU = pd.read_csv('WBID Query.csv')
MO_water = MO_water.loc[MO_water['WBID'].isin(MU['WBID'])]
MO_water.to_file('MO_water.geojson', driver='GeoJSON')   


In [201]:
# MO_water = gpd.read_file("MO_2020_WQS_Lake_Classification_and_Use.geojson", driver='GeoJSON')
# MO_water = MO_water.drop(columns="EVENTDAT").drop(columns="REACHSMDAT").round(0) # Need to drop timestamps for plotting to work
# MU = pd.read_csv('WBID Query.csv')
# MO_water = MO_water.loc[MO_water['WBID'].isin(MU['WBID'])]
# MO_water.to_file('MO_water.geojson', driver='GeoJSON')   
MO_water = gpd.read_file("MO_water.geojson", driver='GeoJSON')
MO_water

,FID,TBLE_NAME,NHD,METADATA,STNDRD_ID,WBID,WBID_TXT,WATER_BODY,WB_CLS,DS_COUNTY,...,SCR_USE,WBCA_USE,WBCB_USE,HHP_USE,NNC,PERM_ID,REACHCODE,Shape__Area,Shape__Length,geometry
0,2,G,Y,N,500,7002,7002.00,Wakonda Lake,L3,Lewis,...,Y,Y,N,Y,N,{380823D7-ADD6-428F-823E-A0BAA0AE542B},07110001002894,551741.0,9629.0,"POLYGON ((-91.51360 40.00006, -91.51379 39.999..."
1,3,G,Y,Y,105,7003,7003.00,Old Lake,L1,Pike,...,Y,N,Y,Y,Y,{46944D6C-A9D1-492F-AE85-2DC7C29198AF},07110004003268,189438.0,3981.0,"POLYGON ((-91.16047 39.34390, -91.16043 39.343..."
2,4,G,Y,N,104,7004,7004.00,Bowling Green Reservoir,L1,Pike,...,Y,N,Y,Y,Y,{D95A5669-7E77-412A-B2D5-24979AAB9FAC},07110004003269,287128.0,5657.0,"POLYGON ((-91.15031 39.33291, -91.15041 39.332..."
3,14,G,Y,N,200,7008,7008.00,Fox Valley Lake,L3,Clark,...,Y,N,Y,Y,Y,{C1B630C9-97EB-436B-AF33-AC06DA113603},07110001012927,735962.0,9134.0,"POLYGON ((-91.76636 40.49566, -91.76693 40.495..."
4,15,G,Y,Y,522,7009,7009.00,Wyaconda Lake,L1,Clark,...,Y,N,Y,Y,Y,{F5ABE43C-6847-472F-A176-1E57DB41F78A},07110001009258,66081.0,1236.0,"POLYGON ((-91.90682 40.39977, -91.90732 40.398..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,3103,G,Y,N,4943,7681,7681.00,North Lake,L3,Callaway,...,Y,N,Y,Y,N,{D630BA1C-A953-4005-A66E-A6C13B6AB9A4},10300200002320,48018.0,1112.0,"POLYGON ((-91.73122 38.98174, -91.73135 38.981..."
407,3108,G,Y,N,471,7297,7297.00,Lac Capri,L3,St. Francois,...,Y,Y,N,Y,Y,{A2B0FCAD-7479-4697-B668-E26FC9238B65},07140104001945,690437.0,5800.0,"POLYGON ((-90.62514 37.89948, -90.62501 37.899..."
408,3109,G,Y,N,4641,7605,7605.00,Lac Carmel,L3,St. Francois,...,Y,Y,N,Y,Y,{F6B39B26-CD55-4630-A754-5ECAFF2FC5E0},07140104001927,360082.0,4844.0,"POLYGON ((-90.64120 37.91900, -90.64131 37.919..."
409,3117,G,Y,N,4650,7614,7614.00,Lac Marseilles,L3,St. Francois,...,Y,Y,N,Y,Y,{351BBDAE-0258-4557-A08C-0168261B32FA},07140104001947,309575.0,3399.0,"POLYGON ((-90.61540 37.90031, -90.61537 37.900..."


### Create new geoJSON for watershed data

In [202]:
MO_watershed = gpd.read_file("MO_water.geojson", driver='GeoJSON')

MO_watershed['Outlet lon'] = 0.0
MO_watershed['Outlet lat'] = 0.0
MO_watershed['Watershed Area'] = 0.0
MO_watershed['Outlet acc'] = 0.0
MO_watershed

,FID,TBLE_NAME,NHD,METADATA,STNDRD_ID,WBID,WBID_TXT,WATER_BODY,WB_CLS,DS_COUNTY,...,NNC,PERM_ID,REACHCODE,Shape__Area,Shape__Length,geometry,Outlet lon,Outlet lat,Watershed Area,Outlet acc
0,2,G,Y,N,500,7002,7002.00,Wakonda Lake,L3,Lewis,...,N,{380823D7-ADD6-428F-823E-A0BAA0AE542B},07110001002894,551741.0,9629.0,"POLYGON ((-91.51360 40.00006, -91.51379 39.999...",0.0,0.0,0.0,0.0
1,3,G,Y,Y,105,7003,7003.00,Old Lake,L1,Pike,...,Y,{46944D6C-A9D1-492F-AE85-2DC7C29198AF},07110004003268,189438.0,3981.0,"POLYGON ((-91.16047 39.34390, -91.16043 39.343...",0.0,0.0,0.0,0.0
2,4,G,Y,N,104,7004,7004.00,Bowling Green Reservoir,L1,Pike,...,Y,{D95A5669-7E77-412A-B2D5-24979AAB9FAC},07110004003269,287128.0,5657.0,"POLYGON ((-91.15031 39.33291, -91.15041 39.332...",0.0,0.0,0.0,0.0
3,14,G,Y,N,200,7008,7008.00,Fox Valley Lake,L3,Clark,...,Y,{C1B630C9-97EB-436B-AF33-AC06DA113603},07110001012927,735962.0,9134.0,"POLYGON ((-91.76636 40.49566, -91.76693 40.495...",0.0,0.0,0.0,0.0
4,15,G,Y,Y,522,7009,7009.00,Wyaconda Lake,L1,Clark,...,Y,{F5ABE43C-6847-472F-A176-1E57DB41F78A},07110001009258,66081.0,1236.0,"POLYGON ((-91.90682 40.39977, -91.90732 40.398...",0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,3103,G,Y,N,4943,7681,7681.00,North Lake,L3,Callaway,...,N,{D630BA1C-A953-4005-A66E-A6C13B6AB9A4},10300200002320,48018.0,1112.0,"POLYGON ((-91.73122 38.98174, -91.73135 38.981...",0.0,0.0,0.0,0.0
407,3108,G,Y,N,471,7297,7297.00,Lac Capri,L3,St. Francois,...,Y,{A2B0FCAD-7479-4697-B668-E26FC9238B65},07140104001945,690437.0,5800.0,"POLYGON ((-90.62514 37.89948, -90.62501 37.899...",0.0,0.0,0.0,0.0
408,3109,G,Y,N,4641,7605,7605.00,Lac Carmel,L3,St. Francois,...,Y,{F6B39B26-CD55-4630-A754-5ECAFF2FC5E0},07140104001927,360082.0,4844.0,"POLYGON ((-90.64120 37.91900, -90.64131 37.919...",0.0,0.0,0.0,0.0
409,3117,G,Y,N,4650,7614,7614.00,Lac Marseilles,L3,St. Francois,...,Y,{351BBDAE-0258-4557-A08C-0168261B32FA},07140104001947,309575.0,3399.0,"POLYGON ((-90.61540 37.90031, -90.61537 37.900...",0.0,0.0,0.0,0.0


In [203]:
MO_water.loc[MO_water['WBID'] == 7113]

,FID,TBLE_NAME,NHD,METADATA,STNDRD_ID,WBID,WBID_TXT,WATER_BODY,WB_CLS,DS_COUNTY,...,SCR_USE,WBCA_USE,WBCB_USE,HHP_USE,NNC,PERM_ID,REACHCODE,Shape__Area,Shape__Length,geometry
73,179,G,Y,N,263,7113,7113.00,King City Old Reservoir,L1,Gentry,...,Y,N,Y,Y,Y,{F5E585ED-B4F2-4A40-B6F2-0A1A78FD2486},10280101002672,54622.0,1242.0,"POLYGON ((-94.51319 40.06433, -94.51279 40.064..."
74,180,G,Y,N,263,7113,7113.00,King City Old Reservoir,L1,Gentry,...,Y,N,Y,Y,Y,{C082121F-E3EC-496D-98D1-01EA4C26CD88},10280101034713,4975.0,276.0,"POLYGON ((-94.51414 40.06220, -94.51450 40.062..."


In [206]:
def delineate_watershed(i):
    
    if  MO_water.iloc[i]['FID'] not in [180, 184, 231, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 244, 247,
                                   248, 249, 252, 255, 256, 257, 258, 259, 312, 315, 316, 317, 318, 320, 323,
                                   324, 326, 327, 328, 329, 330, 332, 333, 334, 335, 338, 339, 340, 341, 342,
                                   344, 346, 347, 350, 351, 352, 354, 355, 356, 357, 360, 379, 827, 3034, 3120,
                                   3117]:

        print(i, MO_water.iloc[i]['FID'], MO_water.iloc[i]['HUC_8'][0:6] )
        stats = zonal_stats(MO_water.iloc[i].geometry, 'hydrosheds_3arc/HydroSHEDS_acc_' + MO_water.iloc[i]['HUC_8'][0:6] + '.tif',
                            stats = ['max'], raster_out = True, geojson_out = True, all_touched=False)

        outlet = pd.DataFrame(columns=list('XY'))
        arr = stats[0]['properties']['mini_raster_array']
        col = np.argmax(np.max(arr, axis=0))
        row = np.argmax(np.max(arr, axis=1))
        c, a, b, f, d, e = stats[0]['properties']['mini_raster_affine'].to_gdal()
        xp = a * col + b * row + a * 0.5 + b * 0.5 + c
        yp = d * col + e * row + d * 0.5 + e * 0.5 + f
        outlet.loc[0] =(xp, yp)

        MO_watershed.iloc[i, MO_watershed.columns.get_loc('Outlet acc')] = (stats[0]['properties']['max'])
        MO_watershed.iloc[i, MO_watershed.columns.get_loc('Outlet lon')] = outlet['X']
        MO_watershed.iloc[i, MO_watershed.columns.get_loc('Outlet lat')] = outlet['Y']    

        grid = Grid.from_raster('hydrosheds_3arc/HydroSHEDS_acc_' + MO_water.iloc[i]['HUC_8'][0:6] + '.tif', nodata=0)
        fdir = grid.read_raster('hydrosheds_3arc/HydroSHEDS_fdir_' + MO_water.iloc[i]['HUC_8'][0:6] + '.tif', nodata=0)
        acc = grid.read_raster('hydrosheds_3arc/HydroSHEDS_acc_' + MO_water.iloc[i]['HUC_8'][0:6] + '.tif', nodata=0)

        x_snap, y_snap =  grid.snap_to_mask(acc >= (stats[0]['properties']['max']), 
                                            (MO_watershed.iloc[i]['Outlet lon'],
                                             MO_watershed.iloc[i]['Outlet lat']))

        catch = grid.catchment(x = x_snap, 
                               y = y_snap, fdir=fdir, xytype='coordinate')  

        # Clip the bounding box to the catchment
        grid.clip_to(catch)
        clipped_acc = grid.view(acc, nodata=np.nan)

        shapes = grid.polygonize()
        catchment_polygon = ops.unary_union([geometry.shape(shape) for shape, value in shapes])

        if catchment_polygon.geom_type == 'MultiPolygon': ## Occasionally will return a multipolygon, so take the largest
            boundary = gpd.GeoSeries(unary_union(catchment_polygon))
            MO_watershed.iloc[i, MO_watershed.columns.get_loc('geometry')] =  boundary
        else:
            MO_watershed.iloc[i, MO_watershed.columns.get_loc('geometry')] = catchment_polygon




In [207]:
for i in range(len(MO_water)):
    delineate_watershed(i)
    
MO_watershed.to_file('MO_watersheds_hydrosheds_3arcsecond.geojson', driver="GeoJSON") 
MO_watershed_utm = MO_watershed.to_crs({'init': 'epsg:3395'})
MO_watershed['Watershed Area'] = MO_watershed_utm.area*1e-6
MO_watershed.to_file('MO_watersheds_hydrosheds_3arcsecond.geojson', driver="GeoJSON")  

0 2 071100
1 3 071100
2 4 071100
3 14 071100
4 15 071100
5 16 071100
6 17 071100
7 18 071100
8 19 071100
9 20 071100
10 23 071100
11 24 071100
12 25 071100
13 26 071100
14 27 071100
15 28 071100
16 29 071100
17 30 071100
18 31 071100
19 32 071100
20 33 071100
21 34 071100
22 35 071100
23 36 071100
24 37 071100
25 39 071100
26 40 071100
27 41 071100
28 42 071100
29 43 071100
30 44 071100
31 46 071100
32 48 071100
33 49 071100
34 50 071100
35 51 071100
36 53 071100
37 58 071100
38 59 071100
39 61 071100
40 62 071100
41 63 071100
42 64 071100
43 65 102400
44 78 102801
45 108 102400
46 109 102801
47 114 102400
48 124 102400
49 125 102400
50 126 102400
51 127 102400
52 133 102400
53 149 103001
54 150 103001
55 151 103001
56 153 103001
57 154 103001
58 155 103001
59 158 103001
60 160 103001
61 161 103001
62 164 103001
63 167 103001
64 168 103001
65 169 103001
66 170 102801
67 171 102801
68 172 102801
69 175 102801
70 176 102801
71 177 102400
72 178 102801
73 179 102801
75 181 102801
76 182 1

/home/gsilsbe/miniconda3/envs/rabpro/lib/python3.9/site-packages/pyproj/crs/crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


## Plot watersheds

In [187]:
MO_watershed = gpd.read_file("MO_watersheds.geojson", driver='GeoJSON')

for i in range(MO_water.shape[0]):
    
    fileout = 'watershed_maps/' +  MO_watershed.iloc[i]['WATER_BODY'] +  ' ' +  str(MO_watershed.iloc[i]['FID']) + '.png'
    
    if not os.path.exists(fileout):

        water_boundary = gpd.GeoSeries(MO_water.iloc[i].geometry)
        watershed_boundary = gpd.GeoSeries(MO_watershed.iloc[i].geometry)

        grid = Grid.from_raster('huc6/huc6_' + MO_water.iloc[i]['HUC_8'][0:6] + '_500.tif', nodata=-9999)
        acc = grid.read_raster('huc6/huc6_acc_' + MO_watershed.iloc[i]['HUC_8'][0:6] + '_500.tif', nodata=-9999)
        dem = grid.read_raster('huc6/huc6_' + MO_watershed.iloc[i]['HUC_8'][0:6] + '_500.tif', nodata=-9999)

        # Grid limits
        lons = [acc.affine[2] + acc.affine[0]*i  for i in range(np.shape(acc.mask)[1])]
        lats = [acc.affine[5] + acc.affine[4]*i  for i in range(np.shape(acc.mask)[0])]

        # Establish plot limits
        delx = watershed_boundary.total_bounds[2] - watershed_boundary.total_bounds[0] 
        dely = watershed_boundary.total_bounds[3] - watershed_boundary.total_bounds[1] 
        minx, maxx = watershed_boundary.total_bounds[0] - 0.5*delx, watershed_boundary.total_bounds[2] + 0.5*delx
        miny, maxy = watershed_boundary.total_bounds[1] - 0.5*dely, watershed_boundary.total_bounds[3] + 0.5*dely

        # Manually set acc outside limits to -9999
        for j in range(np.shape(acc.mask)[0]):
            if lats[j] > maxy or lats[j] < miny:
                acc[j,:] = -9999
        for j in range(np.shape(acc.mask)[1]):
            if lons[j] > maxx or lons[j] < minx:
                acc[:,j] = -9999

        # Clip to limits        
        grid.clip_to(acc)
        acc_view = grid.view(acc)

        fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(8, 4))

        # Upstream Cells
        im1 = ax1.imshow(acc_view,  extent=grid.extent, 
                   cmap='cubehelix',
                   norm=colors.LogNorm(1, acc_view.max()),
                   interpolation='bilinear')
        water_boundary.plot(ax=ax1, color='white')
        watershed_boundary.plot(ax=ax1, color='red', alpha=0.2)
        plt.colorbar(im1, ax=ax1, label='Upstream Cells', fraction=0.046, pad=0.04)

        # Elevation

        # Reset view
        minx, maxx = watershed_boundary.total_bounds[0] - 0.2*delx, watershed_boundary.total_bounds[2] + 0.2*delx
        miny, maxy = watershed_boundary.total_bounds[1] - 0.2*dely, watershed_boundary.total_bounds[3] + 0.2*dely
        # Manually set acc outside limits to -9999
        for j in range(np.shape(dem.mask)[0]):
            if lats[j] > maxy or lats[j] < miny:
                dem[j,:] = -9999
        for j in range(np.shape(dem.mask)[1]):
            if lons[j] > maxx or lons[j] < minx:
                dem[:,j] = -9999
        #grid = Grid.from_raster('huc6/huc6_' + MO_water.iloc[i]['HUC_8'][0:6] + '_500.tif', nodata=-9999)        
        grid.clip_to(dem)
        dem_view = grid.view(dem)

        im2 = ax2.imshow(dem_view,  extent=grid.extent, cmap='cubehelix')
        water_boundary.plot(ax=ax2, color='royalblue')
        watershed_boundary.plot(ax=ax2, color='white', alpha=0.2)
        plt.colorbar(im2, ax=ax2, label='DEM (m)',fraction=0.046, pad=0.04)

        # Extra
        fig.suptitle(MO_watershed.iloc[i]['WATER_BODY'] + ' ' + str(np.round(MO_watershed.iloc[i]['Watershed Area'], decimals=1)) + " km2")
        plt.tight_layout()
        print(fileout)
        fig.savefig(fileout, dpi=300)
        plt.clf()

watershed_maps/Rocky Hollow Lake 154.png
watershed_maps/Watkins Mill Lake 155.png
watershed_maps/Lake Lotawana 158.png
watershed_maps/Odessa Lake 160.png
watershed_maps/Odessa Lake (Old) 161.png
watershed_maps/Longview Lake 164.png
watershed_maps/Lake Jacomo 167.png
watershed_maps/Prairie Lee Lake 168.png
watershed_maps/Lake Tapawingo 169.png
watershed_maps/Jamesport City Lake 170.png
watershed_maps/Jamesport Community Lake 171.png
watershed_maps/Old Bethany City Lake 172.png
watershed_maps/New City Lake 175.png
watershed_maps/Worth County Community Lake 176.png
watershed_maps/Limpp Community State Lake 177.png
watershed_maps/King Lake 178.png
watershed_maps/King City Old Reservoir 179.png
watershed_maps/King City Old Reservoir 180.png
watershed_maps/King City New Reservoir 181.png
watershed_maps/Maysville Lake 182.png


/tmp/ipykernel_3354/3270457482.py:39: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(8, 4))


watershed_maps/Maysville Lake 183.png
watershed_maps/Maysville Lake 184.png
watershed_maps/Buffalo Bill Lake 185.png
watershed_maps/Pony Express Lake 186.png
watershed_maps/Pony Express Lake 187.png
watershed_maps/Pony Express Lake 188.png
watershed_maps/Cameron Lake #3 189.png
watershed_maps/Cameron Lake #1 190.png
watershed_maps/Cameron Lake #2 191.png
watershed_maps/Lake Viking 192.png
watershed_maps/Breckenridge Lake 193.png
watershed_maps/Hamilton Lake 194.png
watershed_maps/Trenton Lake Upper 198.png
watershed_maps/Lake Marie 201.png
watershed_maps/Lake Marie 202.png
watershed_maps/Lake Marie 203.png
watershed_maps/Lake Paho 204.png
watershed_maps/Lake Paho 205.png
watershed_maps/Lake Paho 206.png
watershed_maps/Lake Paho 207.png
watershed_maps/Leisure Lake 209.png
watershed_maps/Crowder St. Park Lake 210.png
watershed_maps/New Marceline City Lake 211.png
watershed_maps/Brookfield Lake 213.png
watershed_maps/Linneus Lake 221.png
watershed_maps/Milan Lake North 222.png
watershed_m

<Figure size 800x400 with 0 Axes>

<Figure size 800x400 with 0 Axes>

...

<Figure size 800x400 with 0 Axes>

<Figure size 800x400 with 0 Axes>

<Figure size 800x400 with 0 Axes>

<Figure size 800x400 with 0 Axes>

<Figure size 800x400 with 0 Axes>